In [36]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
%autoreload 2

In [38]:
from ase.db import connect
from Core import Nanoparticle as NP
from Core import Adsorption as ADS
from MCMC import MonteCarlo
from Core import EnergyCalculator as EC
from Core.LocalEnvironmentFeatureClassifier import TopologicalEnvironmentClassifier
from Core.LocalEnvironmentCalculator import LocalEnvironmentCalculator
import numpy as np

db = connect('/home/g15farris/NiAu/NiAu_database.db')

In [39]:
def get_TFC(top):
    feature_vector = np.zeros(17)
    i = 0
    for f,n in zip([2, 10, 11, 13, 16],[0,24,24,44,48]):
        feature_vector[f] = abs(n-top[i])
        i += 1
    return feature_vector

In [40]:
def get_training_set(db):
    from Core import GlobalFeatureClassifier as GFC
    training_set = []
    for row in db.select():
        p = NP.Nanoparticle()
        p.add_atoms(row.toatoms())
        #GFC.TopologicalFeatureClassifier(p.get_all_symbols()).compute_feature_vector(p)
        p.set_feature_vector('TFC', get_TFC(row.data.TOP))
        p.set_energy('DFT', row.dft_energy)
        training_set.append(p)
    return training_set
training_set = get_training_set(db)

In [41]:
energy_calculator = EC.BayesianRRCalculator('TFC')
energy_calculator.fit(training_set, 'DFT')
coefficents, total_energies = EC.compute_coefficients_for_linear_topological_model(
energy_calculator.get_coefficients(), ['Au', 'Ni'], 140)
energy_calculator.set_coefficients(coefficents)
energy_calculator.set_feature_key('TEC')

Coef symbol_a: Au


In [42]:
start_particle = NP.Nanoparticle()
start_particle.truncated_octahedron(6,1,{'Au':84,'Ni':56})

beta = 200
max_steps = 1000

adsorbate = None
local_feature_classifier = TopologicalEnvironmentClassifier(LocalEnvironmentCalculator()
                                                            , ['Au', 'Ni'])

[best_particle, accepted_energies] = MonteCarlo.run_monte_carlo(
                                            beta, max_steps, start_particle, energy_calculator,
                                            local_feature_classifier, adsorbate
                                                            )

/home/g15farris/NanoParticleLibrary/MCMC/MonteCarlo.py:114: RuntimeWarning: overflow encountered in exp
  acceptance_rate = min(1, np.exp(-beta * delta_e))


Step: 2000
Lowest energy: -242.3930612296039


In [43]:
Au = 0
Ni = 0
Au2 = -0.28120172 
Au2Ni = -1.00039431 
Au3 = -0.35726243
AuNi = -1.01410455
AuNi2 = -2.08995281
Ni2 = -2.23349599
Ni3 = -2.40469383

In [67]:
from MCMC import MonteCarlo as MC
from Core.Nanoparticle import Nanoparticle
from Core.GlobalFeatureClassifier import AdsorptionFeatureVector
from Core.EnergyCalculator import BayesianRRCalculator

best_p = Nanoparticle()
best_p.build_from_dictionary(best_particle)
best_p.construct_neighbor_list()
best_p.construct_adsorption_list()


beta = 200
max_steps = 5000
energy_calculator = BayesianRRCalculator('ADS')
sergey_coefficients = [Au, Ni, Au2, AuNi, Ni2, Au3, Au2Ni, AuNi2, Ni3]
energy_calculator.set_coefficients(sergey_coefficients)
n_adsorbates = 20

[best_particle, accepted_energies] = MC.run_monte_carlo_for_adsorbates(beta, max_steps, best_p, energy_calculator, n_adsorbates)

Step: 2000
Lowest energy: -48.093876599999994
Step: 4000
Lowest energy: -48.093876599999994
Step: 6000
Lowest energy: -48.093876599999994


In [70]:
from ase.visualize import view
from Core.Adsorption import PlaceAddAtoms

#best_p = Nanoparticle()
#best_p.build_from_dictionary(best_particle)
#best_p.construct_neighbor_list()
#best_p.construct_adsorption_list()
ads = PlaceAddAtoms(best_p.get_all_symbols())
ads.bind_particle(best_p)
for f in best_particle['ads']:
    site = list(best_p.get_site_atom_indices(f))
    ads.place_add_atom(best_p, 'O', [site])


view(best_p.get_ase_atoms(), viewer='ngl')

In [66]:
best_p = Nanoparticle()
best_p.build_from_dictionary(best_particle)
best_p.construct_neighbor_list()
best_p.construct_adsorption_list()
AdsorptionFeatureVector(best_p.get_all_symbols()).compute_feature_vector(best_p)
best_p.get_feature_vector('ADS')

array([0, 0, 0, 0, 0, 0, 0, 0, 0])

In [64]:
AdsorptionFeatureVector(best_p.get_all_symbols()).features_type

defaultdict(<function Core.GlobalFeatureClassifier.AdsorptionFeatureVector.__init__.<locals>.<lambda>()>,
            {('Au',): 0,
             ('Ni',): 1,
             ('O',): 2,
             ('Au', 'Au'): 3,
             ('Au', 'Ni'): 4,
             ('Au', 'O'): 5,
             ('Ni', 'Ni'): 6,
             ('Ni', 'O'): 7,
             ('O', 'O'): 8,
             ('Au', 'Au', 'Au'): 9,
             ('Au', 'Au', 'Ni'): 10,
             ('Au', 'Au', 'O'): 11,
             ('Au', 'Ni', 'Ni'): 12,
             ('Au', 'Ni', 'O'): 13,
             ('Au', 'O', 'O'): 14,
             ('Ni', 'Ni', 'Ni'): 15,
             ('Ni', 'Ni', 'O'): 16,
             ('Ni', 'O', 'O'): 17,
             ('O', 'O', 'O'): 18})